# Dico

*Priscilla Kandassamy*

### Import des packages 

In [1]:
from IPython.display import display, clear_output, HTML, Markdown
from requests_html import HTMLSession
from bs4 import BeautifulSoup
session = HTMLSession()
import re
from pylab import specgram, show
import moviepy
from moviepy.editor import AudioFileClip
from IPython import display as audioD
import IPython.display as ipd
from IPython.display import display, clear_output, HTML, Markdown 
import IPython.display as ipd
import ipywidgets as widgets
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None 

### Scrapping 

In [2]:
def scrapping(df):
    path="https://dictionary.cambridge.org/fr/dictionnaire/anglais/"
    listeUK = []
    listeUS = []
    
    for mot in df["motVedette"] :
        lien=path+mot
        response = session.get(lien)
        status = response.status_code
        html_doc = response.content
        
        soup = BeautifulSoup(response.text,'html.parser')
        souplettesBody = soup.find('body')

        soupletteAside = souplettesBody.findAll('amp-audio')
        soupletteAsideuk = soupletteAside[0].find('source')
        soupletteAsideus = soupletteAside[1].find('source')

        rechercher = soup.findAll("div", {"class" : "pos-header dpos-h"})

        mp3uk = soupletteAsideuk['src']
        mp3us = soupletteAsideus['src']

        base_url='https://dictionary.cambridge.org'
        audiouk = base_url+mp3uk
        audious = base_url+mp3us
        
        listeUK.append(audiouk)
        listeUS.append(audious)
                                  
    df['lienUK']=pd.Series(listeUK).values
    df['lienUS']=pd.Series(listeUS).values
    return df

In [3]:
def Audio(df):
    clear_output()
    listeWid = widgets.Output()
    lU = widgets.Output()
    sU = widgets.Output()

    for numLigne in range(len(df)):
        
        labelUK = widgets.Label("UK")
        itemsUK = audioD.Audio(url = df["lienUK"][numLigne])
        spectroUK = df["lienUK"][numLigne]
    
        labelUS = widgets.Label("US")
        itemsUS = audioD.Audio(url = df["lienUS"][numLigne])
        audioUK = widgets.Output()
        audioUK.append_display_data(itemsUK)
        audioUS = widgets.Output()
        audioUS.append_display_data(itemsUS)
        spectroUS = df["lienUS"][numLigne]
        
        lU = widgets.HBox([labelUK,audioUK,labelUS,audioUS])

        labelMot = widgets.Label(df["motVedette"][numLigne])
        labelPron = widgets.Label(str(df["pron"][numLigne]))
        label = widgets.HBox([labelMot, labelPron])


        listeWid.append_display_data(widgets.VBox([label,lU,sU]))
                        
    return listeWid

In [4]:
#Import du dictionnaire
cheminDico = r"C:\Users\kandasp\Downloads\cnam-athon-pronunciation-main (1)\cnam-athon-pronunciation-main\data\dictionary\1757_Buchanan-J.csv" 
dfDicoBuch = pd.read_csv(cheminDico, delimiter=";", encoding="utf8")

dfDicoBuch.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId
7836,1,līfe,NaN,NaN,NaN,n.,30.0,130
8159,3,măʹlănd_e_rs,NaN,NaN,NaN,n.,68.0,134
11905,2,rŭʹmmer,NaN,NaN,NaN,n.,16.0,187
9114,4,nŏnăppearănce,NaN,nbSyll>1 et pas d'accent tonic,NaN,n.,19.0,148
12579,1,ſleek,NaN,NaN,NaN,a.,46.0,196


In [5]:
cheminMap = r"C:\Users\kandasp\Downloads\cnam-athon-pronunciation-main (1)\cnam-athon-pronunciation-main\data\dictionary\Buchanan_PronChar_counts.csv"
dfMap = pd.read_csv(cheminMap, delimiter=";", encoding="utf8")

dfMap.sample(n=5)

,char,occ,graph
7,f,1809,f
38,k,717,k
40,x,406,x
31,g,2604,g
72,ï,1,i


In [6]:
dfMap = dfMap.replace(np.nan,'', regex = True)

In [7]:
def correctionTexte(ancienMot,dfMap):
    Resultat = ""
    
    for lettre in ancienMot: 
        dfRowFinded = dfMap[dfMap['char'] == lettre]
        
        if len(dfRowFinded == 1):
            Resultat += dfRowFinded.iloc[0]['graph']
            
        else:
            Resultat += '?'
            
    return(Resultat)

In [8]:
dfDicoBuch['motVedette'] = dfDicoBuch.apply(lambda x: correctionTexte(x['pron'], dfMap),1)

### 2.1 - Création des colonnes Sampa et Ipa

In [9]:
CheminBigRichSample=r"C:\Users\kandasp\Downloads\cnam-athon-pronunciation-main (1)\cnam-athon-pronunciation-main\data\dataframe\BigiRichSample.csv"
dfBRS = pd.read_csv(CheminBigRichSample, delimiter=";", encoding="utf8")

In [10]:
DfDico = dfDicoBuch.merge(dfBRS, how='inner',left_on= 'motVedette', right_on='hwd')
del DfDico['hwd']
DfDico.sample(n=5)

,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,Sampa,ipa
2346,2,demīʹſe,NaN,NaN,NaN,n.,21.0,67,demise,d i m aI z,dim?z
8423,1,ſno_w_,NaN,NaN,NaN,n.,3.0,198,snow,s n @U,snoʊ
3748,1,gărb,NaN,NaN,NaN,n.,28.0,96,garb,g A r\ b,gɑɹb
9941,4,ŭnwāʹvĕring,NaN,NaN,NaN,a.,31.0,233,unwavering,@ n w eI v 3:r i N,ənw?vɜ:riŋ
8978,2,tăʹlbot,NaN,NaN,NaN,n.,18.0,210,talbot,t { l b @ t,tælbət


In [11]:
DfDico.to_csv(r"C:\Users\kandasp\Downloads\cnam-athon-pronunciation-main (1)\cnam-athon-pronunciation-main\data\dataframe\BuchananRich.csv")

In [12]:
cheminBigi = r"C:\Users\kandasp\Downloads\cnam-athon-pronunciation-main (1)\cnam-athon-pronunciation-main\data\dataframe\BuchananRich.csv"
Bigi = pd.read_csv(cheminBigi, delimiter=",", encoding="utf8")
Bigi.head(3)

,Unnamed: 0,nbSyll,pron,origin,warning,remarques,POS,lineId,pageId,motVedette,Sampa,ipa
0,0,3,aʹbăcus,NaN,NaN,NaN,n.,2.0,19,abacus,{ b @ k @ s,æbəkəs
1,1,3,ābăʹndon,F.,NaN,NaN,v.,5.0,19,abandon,@ b { n d @ n,əbændən
2,2,4,ābăʹndŏned,NaN,NaN,NaN,a.,6.0,19,abandoned,@ b { n d @ n d,əbændənd


In [13]:
dfBigi=Bigi[["motVedette","nbSyll","pron","Sampa","ipa"]]
dfBigi.head(3)

,motVedette,nbSyll,pron,Sampa,ipa
0,abacus,3,aʹbăcus,{ b @ k @ s,æbəkəs
1,abandon,3,ābăʹndon,@ b { n d @ n,əbændən
2,abandoned,4,ābăʹndŏned,@ b { n d @ n d,əbændənd


In [14]:
filtreHWD = widgets.Text(value="",description='filtre hwd',layout={'width': '250px'} )
filtreIPA = widgets.Text(value="",description='filtre ipa',layout={'width': '250px'} )
occMax = widgets.IntSlider(min=0,max=10,step=1,description='Occ.Max:',value=1)
bouton = widgets.Button(description='Démarrer', icon = "check")

In [15]:
def on_button_clicked(_):
    nbLigne = occMax.value
    HWD = filtreHWD.value 
    IPA = filtreIPA.value
    df = ""

    with dataFrame:
        clear_output()
        if (HWD != ""):
            df = dfBigi[(dfBigi.motVedette.str.match(HWD))]
            if(IPA != ""):
                df = df[(df.pron.str.match(IPA))]
        elif (IPA != ""):
            df = dfBigi[(dfBigi.pron.str.match(IPA))]

        if(len(df) > nbLigne):
            df = df.sample(n=nbLigne).copy()
        df = df.reset_index()    
        display(df)
        
    with audio :
        clear_output()
        df = scrapping(df)  
        
        listeAudio = Audio(df) 
        display(listeAudio)
        
            
bouton.on_click(on_button_clicked)

In [16]:
dataFrame = widgets.Output()
audio = widgets.Output()

In [17]:
mod1 = widgets.HBox([filtreHWD, filtreIPA, occMax, bouton])
mod2 = widgets.HBox([dataFrame])
mod3 = widgets.HBox([audio])

interface = widgets.VBox([mod1, mod2, mod3])
interface